This notebook demonstrates basic usage of the firefly_client API.

Note that it may be necessary to wait for some cells (like those displaying an image) to complete before executing later cells.

## Setup

Imports for Python 2/3 compatibility

In [ ]:
from __future__ import print_function, division, absolute_import

Imports for firefly_client

In [ ]:
from firefly_client import FireflyClient

In this example we use the irsaviewer application for the server.

In [ ]:
host='irsa.ipac.caltech.edu:80'
channel = 'myChannel8'
basedir = 'irsaviewer'

fc = FireflyClient(host, channel=channel, basedir=basedir)

## Download some data

Download a sample image and table.

We use astropy here for convenience, but firefly_client itself does not depend on astropy.

In [ ]:
import astropy.utils.data

Download a cutout of a WISE band 1 image.

In [ ]:
image_url = ('http://irsa.ipac.caltech.edu/ibe/data/wise/allsky/4band_p1bm_frm/6a/02206a' + 
             '/149/02206a149-w1-int-1b.fits?center=70,20&size=200pix')
filename = astropy.utils.data.download_file(image_url, cache=True, timeout=120)

Download a 2MASS catalog

In [ ]:
table_url = ("http://irsa.ipac.caltech.edu/TAP/sync?FORMAT=IPAC_TABLE&" +
                                             "QUERY=SELECT+*+FROM+fp_psc+WHERE+CONTAINS(POINT('J2000',ra,dec)," +
                                             "CIRCLE('J2000',70.0,20.0,0.1))=1")
tablename = astropy.utils.data.download_file(table_url, timeout=120, cache=True)

## Display tables and catalogs

Open a browser to the firefly server in a new tab. Only works when running the notebook locally.

In [ ]:
fc.launch_browser()

Alternatively, uncomment the lines below to display the browser application in the notebook.

In [ ]:
#from IPython.display import IFrame
#print('url: %s' % fc.get_firefly_url())
#IFrame(fc.get_firefly_url(), 1100, 1000)

A local file can be uploaded to the server and then displayed.

In [ ]:
imval = fc.upload_file(filename)
status = fc.show_fits(file_on_server=imval, plot_id="wise-cutout", title='WISE Cutout')

`show_fits` can also display an image from a URL

In [ ]:
status = fc.show_fits(file_on_server=None, plot_id="wise-fullimage", 
                      URL='http://irsa.ipac.caltech.edu/ibe/data/wise/allsky/4band_p1bm_frm/6a/02206a' + 
             '/149/02206a149-w1-int-1b.fits')

A catalog (with coordinates) will overlay the image by default.

In [ ]:
file= fc.upload_file(tablename)
status = fc.show_table(file, tbl_id='tablemass', title='My 2MASS Catalog', page_size=50)

An xy plot can be added using the uploaded table data.

In [ ]:
status = fc.show_xyplot(tbl_id='tablemass', xCol='j_m', yCol='h_m-k_m')

Zoom the full image.

In [ ]:
status = fc.set_zoom('wise-fullimage', 2)

Pan the image to a celestial coordinate.

In [ ]:
status = fc.set_pan('wise-fullimage', 70, 20, coord='J2000')

Set the stretch

In [ ]:
status = fc.set_stretch('wise-fullimage', stype='zscale', algorithm='linear')

Add region data

In [ ]:
my_regions= ['image;polygon 125 25 160 195 150 150 #color=cyan',
            'icrs;circle 69.95d 20d 30i # color=orange text={region 5/7}']
status = fc.add_region_data(region_data=my_regions, region_layer_id='layer1',
                  plot_id='wise-cutout')

Remove the second region

In [ ]:
fc.remove_region_data(region_data=my_regions[1], region_layer_id='layer1')